# Classify Emails beyond spams

# LDA - Latent Dirichlet Allocation


### This notebook shows the implementation of LDA on the data + Synonyms. Level 2
### Final Project - Riti Chakraborty¶

In [1]:
#Riti Chakraborty

#importing the required libraries
import pandas as pd
import numpy as np
import random
from sklearn.feature_extraction.text import CountVectorizer
import numpy
from numpy import nan

#for flattening lists
from itertools import chain

#To handle warning
import warnings
warnings.filterwarnings('ignore')

#For implementing Natural Language Processing approaches.
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer

#For using Regular expression
import re

#For Handling Strings
import string

#For implementing word sense disambiguation
from nltk.corpus import wordnet as wn
from wordsegment import load, segment

#Important to call load()
load()

#For LDA Implementation # Importing Gensim
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
import gensim
from gensim import corpora

#For evaluation Topic models formed
from gensim.models import CoherenceModel

#For visualising LDA Output
import pyLDAvis
import pyLDAvis.gensim 


In [2]:
#Reading the data exported from the previous file after initial preprocessing
data_subset=pd.read_csv("../exported_tables/data_subset.csv")
#converting the data in the dataframe into str type
data_subset = data_subset.astype(str) 


#retaining proper index
data_subset=data_subset.reset_index(drop=True)

num_of_rows=len(data_subset.index)
print("Total Number of rows: ",num_of_rows)

# Converting each row to list of lists
list1=data_subset.values.tolist()
list2=[]

for i in range(0, len(list1)):
    list2.append(' '.join(map(str, list1[i])))

#converting each row into vectors #Printing the count of each term in the emails #Emails on Rows and Terms on columns
vectorizer1 = CountVectorizer()
row_vectors=vectorizer1.fit_transform(list2).todense()


Total Number of rows:  1135


### Defining a Function for removing Stopwords

In [3]:
def stopword_remove(l2):
    fil_list2=[]
    for sent in l2:
        stop_words=["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "spam","yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "hibody", "body","these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]
        word_tokens = word_tokenize(sent.lower())
        filtered_sentence = [w for w in word_tokens if not w in stop_words]
        filtered_sentence = []

        for w in word_tokens:
            if w not in stop_words:
                filtered_sentence.append(w)

        fil_list2.append(' '.join(filtered_sentence))
    return fil_list2


#Calling the function to remove stop words
fil_list2=stopword_remove(list2)

### Defining a function for punctuation removal

In [4]:
#Defining a function to remove punctuation
def no_punctuation(my_str):
    punctuations = '''!()-[]{};:'"\\,<>./?@#$%^&*_~'''
    no_punct = ""
    for char in my_str:
        if char not in punctuations:
            no_punct = no_punct + char
        else:
            no_punct = no_punct + " "

# display the unpunctuated string
    return no_punct

### Defining function for word tokenization, segmentation and addition of Synonyms from the Wordnet API

In [5]:
def seg_syn(l1):
    wordlist2=[]
    wordlist2.append(no_punctuation(l1).split())
    seg=[]
    for w in wordlist2:
        no_integers = [x for x in w if not (x.isdigit() or x[0] == '-' and x[1:].isdigit())]
        pure_string=[x for x in no_integers if not any(c.isdigit() for c in x)]
        for s in pure_string:
            seg.append(segment(s))
    flat_seg_list = [item for sublist in seg for item in sublist]
    d = {'Term':[], 'Synonyms':[], 'Hypernyms':[]}
    d1={}
    syn0=[]
    hyp0=[]
    for f in flat_seg_list:
#         print(f)
        d['Term'].append(f)        
        for syn in wn.synsets(f):
            synsets=syn
#             print("Hello")
            for l in syn.lemmas():
                syn0.append(l.name())
                d1[f]=l.name()
                d['Synonyms'].append(l.name())

    return syn0,flat_seg_list
#     return flat_seg_list


In [6]:

list2_syn=[]
for l in fil_list2:

    #Storing the synonyms and terms returned from the previous function
    syno,termz=seg_syn(str(l))
    list2_syn.append(syno+termz)

#flattening the list created from above
flat_list_syn_hyp=len(list(chain(*list2_syn)))

#Converting back to strings
flat_list2_syn=[]
for ls in list2_syn:
    flat_list2_syn.append(' '.join(list(set(ls))))

df_flatlist=pd.DataFrame(flat_list2_syn)
df_flatlist["Index"]=df_flatlist.index

doc_compt=[]
for text in flat_list2_syn:
    doc_compt.append(re.sub(r'\b\w{1,4}\b', '', str(text)))


### LDA on the data

In [11]:
random.seed(3425)
stop = set(stopwords.words('english'))
exclude_punct = set(string.punctuation) 
lemma = WordNetLemmatizer()

def clean(doc):
    
    #Removing Stop words
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    
    #Removing Punctuation
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude_punct)
    
    #for stemming 
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    result = ''.join([i for i in normalized if not i.isdigit()])
    
    return result

doc_clean = [clean(doc).split() for doc in  doc_compt]        

#Creating a dictionary
dictionary = gensim.corpora.Dictionary(doc_clean)

#Creating a bag of words
bow_corpus = [dictionary.doc2bow(doc) for doc in doc_clean]

#LDA model
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=5, id2word=dictionary, passes=2, workers=2)
for idx, topic in lda_model.print_topics(-1):
    print('\n Topic: {} \nWords: {}'.format(idx, topic))        




 Topic: 0 
Words: 0.020*"atomicnumber" + 0.011*"granny" + 0.011*"grandma" + 0.011*"nanriver" + 0.010*"nanna" + 0.010*"grandmother" + 0.010*"grannie" + 0.006*"friday" + 0.005*"speedoflight" + 0.005*"nosecandy"

 Topic: 1 
Words: 0.010*"nanna" + 0.010*"grannie" + 0.010*"grandmother" + 0.009*"grandma" + 0.009*"nanriver" + 0.009*"granny" + 0.007*"atomicnumber" + 0.005*"profit" + 0.004*"tuesday" + 0.004*"thursday"

 Topic: 2 
Words: 0.010*"state" + 0.006*"atomicnumber" + 0.006*"grandmother" + 0.006*"grandma" + 0.006*"nanriver" + 0.006*"grannie" + 0.006*"nanna" + 0.005*"granny" + 0.005*"tuesday" + 0.004*"august"

 Topic: 3 
Words: 0.006*"grannie" + 0.006*"grandma" + 0.006*"grandmother" + 0.006*"nanriver" + 0.006*"granny" + 0.006*"nanna" + 0.005*"atomicnumber" + 0.005*"thursday" + 0.003*"wednesday" + 0.003*"sunday"

 Topic: 4 
Words: 0.013*"atomicnumber" + 0.009*"grannie" + 0.009*"nanna" + 0.008*"grandmother" + 0.008*"nanriver" + 0.008*"granny" + 0.008*"grandma" + 0.004*"wednesday" + 0.004*"

### Computing Perplexity and Coherence Score

In [12]:
random.seed(3425)
# Compute Perplexity : lowest perplexity is considered the best.
perplexity=lda_model.log_perplexity(bow_corpus)
print('\n Perplexity of the Spam Classification model: ', perplexity)  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=doc_clean, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\n Coherence Score of the Spam Classification model: ', coherence_lda)


 Perplexity of the Spam Classification model:  -8.22334263670747

 Coherence Score of the Spam Classification model:  0.43922060642514227


# Visualization

In [13]:
pyLDAvis.enable_notebook()
id2word=dictionary
vis = pyLDAvis.gensim.prepare(lda_model, bow_corpus, id2word)
vis

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
0      30.901205        1       1 -0.078711  0.007591
4      21.431030        1       2 -0.023283 -0.025541
1      17.447346        1       3  0.007258 -0.031474
3      15.679893        1       4  0.065308 -0.034088
2      14.540524        1       5  0.029427  0.083512, topic_info=     Category        Freq                           Term        Total  \
term                                                                    
47    Default  184.000000                          state   184.000000   
7     Default   76.000000                        connect    76.000000   
60    Default   67.000000                         united    67.000000   
59    Default   76.000000                          unite    76.000000   
26    Default   81.000000                         linkup    81.000000   
48    Default   57.000000                statedepartment    57.000000   
39    Default   58.000000                     putforward    58.000000   
5     Default   70.000000                        combine    70.000000   
14    Default   57.000000              departmentofstate    57.000000   
31    Default   57.000000                         nation    57.000000   
12    Default   59.000000                        country    59.000000   
58    Default   68.000000                          unify    68.000000   
52    Default   62.000000                         submit    62.000000   
49    Default   55.000000                  stateofmatter    55.000000   
23    Default   67.000000                         joined    67.000000   
16    Default   63.000000                        express    63.000000   
61    Default   57.000000  unitedstatesdepartmentofstate    57.000000   
36    Default   60.000000                          posit    60.000000   
6     Default   65.000000                   commonwealth    65.000000   
2     Default   57.000000                    bodypolitic    57.000000   
30    Default   69.000000                          merge    69.000000   
43    Default   57.000000                     respublica    57.000000   
38    Default   60.000000                       province    60.000000   
594   Default  253.000000                        coulomb   253.000000   
611   Default  253.000000                       vitaminc   253.000000   
598   Default  253.000000               degreecentigrade   253.000000   
605   Default  254.000000                      nosecandy   254.000000   
591   Default  255.000000                        century   255.000000   
595   Default  254.000000                       cytosine   254.000000   
57    Default  287.000000                        tuesday   287.000000   
...       ...         ...                            ...          ...   
2      Topic5   37.532307                    bodypolitic    57.491833   
43     Topic5   37.477135                     respublica    57.469090   
36     Topic5   38.937836                          posit    60.108635   
16     Topic5   40.959198                        express    63.469521   
5      Topic5   44.777061                        combine    70.563828   
58     Topic5   43.086712                          unify    68.045158   
7      Topic5   47.557423                        connect    76.083458   
23     Topic5   42.152378                         joined    67.197578   
38     Topic5   37.522934                       province    60.060608   
59     Topic5   46.242805                          unite    76.142555   
30     Topic5   41.291286                          merge    69.806404   
6      Topic5   39.012516                   commonwealth    65.765900   
26     Topic5   46.863586                         linkup    81.607574   
3413   Topic5    7.855384                       computer    12.317396   
3425   Topic5    7.351267    informationprocessingsystem    11.431904   
1328   Topic5    8.159752                     rightfield    13.203217   
80